# База данных

### Преамбула

Представим, что мы занимаемся разработкой высоконагруженного приложения. Большинство высоконагруженных приложений сегодня не обходятся без баз данных. С развитием микрисервисной архитектуры прямой доступ к базе данных стал считаться мовитоном, для этих целей обычно используются обертки вокруг баз данных, чтобы обеспечить безопасный доступ к данным нескольких сервисов. Нашей целью будет реализовать примитивную обертку.

### Детальное описание

Наше приложение, без привязки к какому-либо домену, предоставляет пользователю некоторые услуги. Чтобы получить доступ к этим услугам, пользователь должен зарегистрироваться. Во время регистрации пользователь должен придумать уникальный никнейм, придумать надежный пароль, привязать адрес своей электронной почты, указать дату рождения. Причем к предоставляемой информации существует ряд требований:

- никнейм должен быть уникальный в контексте нечувствительности к кейсу;  
- никнейм должен состоять не менее чем из 2 и не более чем из 10 допустимых символов; допустимыми символами считаются буквы английского алфавита в верхнем и нижнем регистре, а также цифры от 0 до 9; никнейм не может начинаться с цифры;  
- пароль может состоять только из английских буквы в верхнем и нижнем регистре, цифр от 0 до 9 и из знаков пунктуации;  
- пароль считается надежным, если его длина не меньше 8 символов; также пароль должен содержать цифры, английские буквы в верхнем и нижнем регистре, знаки препинания;  
- почта должна быть уникальна - не разрешается привязывать более одного аккаунта к одному и тому же адресу;  
- пользователь должен быть совершеннолетним, т.е. с его даты рождения должно пройти не менее 18 лет;  

Если никнейм, пароль, почта или возраст не соответствуют установленным требованиям, регистрация пользователя заканчивается возбуждением исключения `ValueError`, в котором сообщается, что именно не соответсвует требованиям. Иначе, записи о пользователе присваивается уникальный идентефикатор, который используется микросервисами приложения для обмена данными. В базу данных заносится следующая информация:

- уникальный идентефикатор;  
- никнейм;  
- пароль;  
- электронная почта;  
- дата рождения;  
- время последней активности в приложении - сначала это дата регистрации;  

Визуализация данных:

|id|nickname|password|email|birthday|last action timestamp|
|--|--|--|--|--|--|
|1 |Alex| 1a2Bc!ef| alex@gmail.com| 2001-09-10 | 2023-10-13

Существуют следующие сценарии взаимодействия микросервисов нашего приложения с данной базой данных:

- добавление новой записи;  
- запрос информации о записях по указанным индентификаторам;  
- изменение никнейма, пароля или электронной почты по указанному идентефикатору; причем новые значения должны удовлетворять всем ограничениям, накладываемым на соответствующие данные;  
- удаление записи из БД по указанному id;  
- обновление поля last action timestamp по id;  

Все действия, кроме удаления записи, сопровождаются обновлениям поля `last action timestamp` - запись текущего таймстемпа.

Также, поскольку наше приложение - это стартап и мы не можем позволить себе вечное хранение огромного количества данных, наша обертка должна обладать функционалом для удаления данных пользователей, непроявляющих активность в течении полугода. Эта функция будет использоваться некторым внешним скедулером, передающим нам таймстемп относительно которого и вычисляются эти 6 месяцев. 

**Опционально:**

- предусмотреть возможность бекапа - сохранение данных нашей питонячьей базы в текстовый файл;  
- предусмотреть возможность восстановления нашей питонячьей базы данных из текстового файла;  

-------------------------------------
**Реализация**

In [6]:
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta
from typing import Iterable, Any
from enum import Enum

from string import (
    ascii_lowercase,
    ascii_uppercase,
    punctuation,
    digits,
)

In [7]:
class UserFields(Enum):
    nickname = 0
    password = 1
    email = 2
    birthday = 3
    last_action_ = 4

In [8]:
id_gen = 0

data_base = {}
names = set()
emails = set()

In [9]:
def is_valid_nickname(nickname: str) -> bool:
    global names

    if not isinstance(nickname, str):
        raise TypeError(
            f'Unexpected nickname type: {type(nickname).__name__}; '
            'str type was expected;'
        )

    nickname = nickname.lower()

    return all([
        2 <= len(nickname) <= 10,
        nickname.isalnum(),
        not nickname[0].isdigit(),
        nickname not in names,
    ])

In [10]:
def is_valid_password(password: str) -> bool:
    if not isinstance(password, str):
        raise TypeError(
            f'Unexpected password type: {type(password).__name__}; '
            'str type was expected;'
        )
    
    password_set = set(password)

    return all([
        8 <= len(password),
        password_set & set(ascii_lowercase),
        password_set & set(ascii_uppercase),
        password_set & set(digits),
        password_set & set(punctuation),
    ])

In [ ]:
def is_valid_birthday(birthday: datetime):
    current_date = datetime.now()
    shifted_date = datetime(
        datetime.now().year - 18, current_date.month, current_date.day
    )

    return birthday <= shifted_date

In [ ]:
def add_new_user(description: list[Any]) -> None:
    global id_gen, data_base, emails

    if not isinstance(description, list):
        raise TypeError(
            f'Unexpected description type: {type(description).__name__}; '
            'list type was expected;'
        )
    
    if len(description) != 4:
        raise TypeError(
            f'Incorrect description len: {len(description)}; ')
    
    if not is_valid_nickname(description[UserFields.nickname.value]):
        raise ValueError(
            f'Invalid nickname: {description[UserFields.nickname.value]}')

    if not is_valid_password(description[UserFields.password.value]):
        raise ValueError(
            f'Invalid password: {description[UserFields.password.value]}')
    
    if description[UserFields.email.value] in emails:
        raise ValueError(
            f'{description[UserFields.email.value]} already exists')
    
    base_date = datetime()

    if age <= 

In [11]:
is_valid_password()

True